In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale, minmax_scale, robust_scale

#scaling
from sklearn.preprocessing import StandardScaler
#데이터 분할
from sklearn.model_selection import train_test_split
#로지스틱 회귀
from statsmodels.api import Logit
#평가함수
from sklearn.metrics import accuracy_score
#confusion matrix
from sklearn.metrics import confusion_matrix
#설명변수 데이터 생성
#df1_x = df1.drop(['new_judge',''], axis = 1)
from statsmodels.api import qqplot, add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

##insu_pre_review 와 insu_risk customer id 기준으로 merge

In [2]:
df1 = pd.read_csv('./DATA/insu_pre_review.csv', encoding = 'EUC-KR')
df1

,customer_id,review_cat,judge,review_date,gender,age,height,weight,bust,waist,...,bt_mch_judge,bt_mchc_judge,bt_mvc_judge,bt_plat_judge,bt_rbc_judge,bt_wbc_judge,bt_rgpt_judge,bt_sgot_judge,bt_sgpt_judge,bt_trig_judge
0,C112346,혈액검사,재검,2014-10-09,1,70,165,63,92.0,88.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,C112350,일반검진,재검,2014-11-18,1,69,178,75,99.0,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C112355,일반검진,재검,2015-03-18,1,70,169,69,96.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C112356,혈액검사,재검,2015-09-28,1,70,165,64,103.0,92.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C112358,혈액검사,거절,2015-02-24,1,70,166,73,100.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14933,C134246,혈액검사,승인,2015-02-11,2,63,150,53,94.0,80.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
14934,C134250,일반검진,승인,2015-09-16,2,52,164,60,90.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14935,C134251,일반검진,재검,2015-06-21,2,57,157,48,89.0,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14936,C134252,혈액검사,재검,2015-06-26,2,66,150,47,86.0,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df2 =  pd.read_csv('./DATA/insu_risk.csv', encoding = 'EUC-KR')
df2 = df2.drop('Unnamed: 0', axis = 1)

In [7]:
df2

,customer_id,req_id,insu_req_amount,insu_pay_amount,insu_cum_amount,insu_duration,customer_cost,risk
0,C112379,C112379-20110314-17403-01-001,623098,1214577.0,1800000,15,-0.949255,1
1,C112404,C112404-20100831-12754-01-001,808750,1008750.0,600000,5,-0.247295,1
2,C112411,C112411-20101111-18755-01-001,333680,553680.0,240000,2,-0.659314,1
3,C112570,C112570-20130716-13737-01-001,2762279,2852279.0,4875000,39,-0.032582,1
4,C112856,C112856-20120717-16134-02-001,234820,314820.0,2875000,23,-0.340686,1
...,...,...,...,...,...,...,...,...
2078,C133614,C133614-20121220-18985-01-001,262570,252570.0,6380000,29,0.038085,0
2079,C133633,C133633-20130806-12110-01-001,325600,296509.0,8360000,38,0.089346,0
2080,C133640,C133640-20121024-13962-01-001,92624,33812.0,6380000,29,0.634954,0
2081,C133760,C133760-20120813-15856-01-001,4808179,2529090.0,1080000,27,0.474003,0


In [8]:
df3 = pd.DataFrame(df2[['customer_id','risk']])
df3

,customer_id,risk
0,C112379,1
1,C112404,1
2,C112411,1
3,C112570,1
4,C112856,1
...,...,...
2078,C133614,0
2079,C133633,0
2080,C133640,0
2081,C133760,0


In [9]:
df_1 = pd.merge(df1, df3, on = 'customer_id')

In [10]:
df_1

,customer_id,review_cat,judge,review_date,gender,age,height,weight,bust,waist,...,bt_mchc_judge,bt_mvc_judge,bt_plat_judge,bt_rbc_judge,bt_wbc_judge,bt_rgpt_judge,bt_sgot_judge,bt_sgpt_judge,bt_trig_judge,risk
0,C112379,혈액검사,재검,2014-12-09,1,69,160,59,84.0,72.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
1,C112404,혈액검사,재검,2015-03-29,1,69,175,79,101.0,92.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,C112411,일반검진,재검,2015-09-20,1,69,165,70,92.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,C112570,일반검진,재검,2015-05-27,1,67,165,75,95.0,89.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,C112856,혈액검사,재검,2015-08-04,1,62,165,70,96.0,82.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,C134184,혈액검사,승인,2015-08-13,2,53,155,62,94.0,82.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2079,C134188,일반검진,승인,2015-06-22,1,41,175,78,100.0,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2080,C134197,혈액검사,재검,2014-12-23,1,42,178,70,95.0,86.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2081,C134221,혈액검사,재검,2015-09-09,2,51,159,52,85.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [12]:
df_1.to_csv('./DATA/insu_result.csv',encoding = 'euckr')